In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("test") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/26 08:10:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df= spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

25/06/26 08:11:05 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [3]:
from pyspark.sql.functions import explode, split

In [4]:
words = df.select(explode(split(df.value, " ")).alias("word"))

In [5]:
words.explain()

== Physical Plan ==
*(1) Generate explode(split(value#0,  , -1)), false, [word#3]
+- StreamingRelation socket, [value#0]




In [6]:
words.explain(True)

== Parsed Logical Plan ==
'Project ['explode('split(value#0,  , -1)) AS word#1]
+- ~StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.TextSocketSourceProvider@3672fdc3, socket, org.apache.spark.sql.execution.streaming.sources.TextSocketTable@11167dcf, [host=localhost, port=9999], [value#0]

== Analyzed Logical Plan ==
word: string
~Project [word#4]
+- ~Generate explode(split(value#0,  , -1)), false, [word#4]
   +- ~StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.TextSocketSourceProvider@3672fdc3, socket, org.apache.spark.sql.execution.streaming.sources.TextSocketTable@11167dcf, [host=localhost, port=9999], [value#0]

== Optimized Logical Plan ==
~Generate explode(split(value#0,  , -1)), [0], false, [word#4]
+- ~StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.TextSocketSourceProvider@3672fdc3, socket, org.apache.spark.sql.execution.streaming.sources.TextSocketTable@11167dcf, [host=localhost, port=9999], [value#0]

== Physical 

In [8]:
# query = words.writeStream \
#     .outputMode("append") \
#     .format("csv") \
#     .option("path", "./output/wordcount") \
#     .option("checkpointLocation", "./checkpoint/wordcount") \
#     .start()

# query.awaitTermination()

In [7]:
query = words.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

25/06/26 08:11:20 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ff1b32da-179c-466c-a2bd-a74cca208ea8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/06/26 08:11:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+
|word|
+----+
+----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
| word|
+-----+
|hello|
+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+----+
|word|
+----+
|  hi|
|  hi|
|  hi|
+----+

-------------------------------------------
Batch: 3
-------------------------------------------
+-------+
|   word|
+-------+
|aaditya|
+-------+

-------------------------------------------
Batch: 4
-------------------------------------------
+-------+
|   word|
+-------+
|aaditya|
|     ji|
+-------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [8]:
query = words.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

25/06/26 08:14:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-818fbc70-7bb0-4215-8edd-0ecd89f8a1e3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/06/26 08:14:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AnalysisException: [STREAMING_OUTPUT_MODE.UNSUPPORTED_OPERATION] Invalid streaming output mode: complete. This output mode is not supported for no streaming aggregations on streaming DataFrames/DataSets. SQLSTATE: 42KDE

In [9]:
spark.stop()